In [ ]:
# Audio Quality Metrics: SNR, VAD, Empty Segments

In [1]:
import numpy as np
import librosa
import soundfile as sf
import os

def calculate_snr(signal, noise):
    """
    Calculate Signal-to-Noise Ratio (SNR) in dB.
    
    Parameters:
    -----------
    signal : array-like
        The clean signal
    noise : array-like
        The noise signal (must be same length as signal)
    
    Returns:
    --------
    snr_db : float
        SNR in decibels
    """
    # Ensure arrays are numpy arrays and same length
    signal = np.array(signal)
    noise = np.array(noise)
    
    if len(signal) != len(noise):
        raise ValueError("Signal and noise must have the same length")
    
    # Calculate power (mean squared value)
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    
    # Avoid division by zero
    if noise_power == 0:
        return np.inf
    
    # Calculate SNR in dB
    snr_db = 10 * np.log10(signal_power / noise_power)
    
    return snr_db

def calculate_snr_from_files(clean_file, noisy_file):
    """
    Calculate SNR between two audio files.
    
    Parameters:
    -----------
    clean_file : str
        Path to clean/reference audio file
    noisy_file : str
        Path to noisy audio file
    
    Returns:
    --------
    snr_db : float
        SNR in decibels
    """
    # Load audio files
    clean, sr_clean = librosa.load(clean_file, sr=None)
    noisy, sr_noisy = librosa.load(noisy_file, sr=None)
    
    # Resample if sample rates differ
    if sr_clean != sr_noisy:
        noisy = librosa.resample(noisy, orig_sr=sr_noisy, target_sr=sr_clean)
    
    # Trim to same length
    min_len = min(len(clean), len(noisy))
    clean = clean[:min_len]
    noisy = noisy[:min_len]
    
    # Calculate noise as difference
    noise = noisy - clean
    
    # Calculate SNR
    snr_db = calculate_snr(clean, noise)
    
    return snr_db

def detect_noise_segments(signal, sr, frame_length=2048, hop_length=512, energy_threshold_percentile=20):
    """
    Detect noise-only segments in an audio signal using energy-based Voice Activity Detection (VAD).
    
    Parameters:
    -----------
    signal : array-like
        Audio signal
    sr : int
        Sample rate
    frame_length : int
        Frame length for analysis
    hop_length : int
        Hop length for analysis
    energy_threshold_percentile : float
        Percentile to use as threshold (lower = more sensitive to noise)
    
    Returns:
    --------
    noise_mask : array
        Boolean mask indicating noise segments
    """
    signal = np.array(signal)
    frame_energy = []
    
    # Calculate frame energy
    for i in range(0, len(signal) - frame_length + 1, hop_length):
        frame = signal[i:i + frame_length]
        energy = np.mean(frame ** 2)
        frame_energy.append(energy)
    
    frame_energy = np.array(frame_energy)
    
    # Use percentile as threshold (lower energy segments are likely noise)
    threshold = np.percentile(frame_energy, energy_threshold_percentile)
    
    # Identify noise frames
    noise_frames = frame_energy < threshold
    
    # Convert frame-level mask to sample-level mask
    noise_mask = np.zeros(len(signal), dtype=bool)
    for i, is_noise in enumerate(noise_frames):
        start = i * hop_length
        end = min(start + frame_length, len(signal))
        noise_mask[start:end] = is_noise
    
    return noise_mask

def estimate_noise_from_signal(signal, sr, method='vad', **kwargs):
    """
    Estimate noise characteristics from a single audio signal.
    
    Parameters:
    -----------
    signal : array-like
        Audio signal
    sr : int
        Sample rate
    method : str
        Method to use: 'vad' (Voice Activity Detection) or 'spectral_subtraction'
    **kwargs
        Additional parameters for noise detection
    
    Returns:
    --------
    noise_estimate : array
        Estimated noise signal
    noise_mask : array
        Boolean mask indicating noise segments
    """
    signal = np.array(signal)
    
    if method == 'vad':
        # Use VAD to detect noise segments
        noise_mask = detect_noise_segments(signal, sr, **kwargs)
        
        # Estimate noise from detected segments
        if np.any(noise_mask):
            # Use noise segments directly
            noise_estimate = signal.copy()
            noise_estimate[~noise_mask] = 0  # Zero out speech segments
        else:
            # If no noise segments found, use minimum energy approach
            frame_length = kwargs.get('frame_length', 2048)
            hop_length = kwargs.get('hop_length', 512)
            frame_energy = []
            for i in range(0, len(signal) - frame_length + 1, hop_length):
                frame = signal[i:i + frame_length]
                energy = np.mean(frame ** 2)
                frame_energy.append(energy)
            frame_energy = np.array(frame_energy)
            min_energy_idx = np.argmin(frame_energy)
            start = min_energy_idx * hop_length
            end = min(start + frame_length, len(signal))
            noise_estimate = np.zeros_like(signal)
            noise_estimate[start:end] = signal[start:end]
            noise_mask = np.zeros(len(signal), dtype=bool)
            noise_mask[start:end] = True
    
    elif method == 'spectral_subtraction':
        # Estimate noise from spectral minimums (assumes stationary noise)
        stft = librosa.stft(signal)
        magnitude = np.abs(stft)
        
        # Estimate noise spectrum as minimum over time
        noise_spectrum = np.min(magnitude, axis=1, keepdims=True)
        
        # Reconstruct noise signal
        noise_stft = noise_spectrum * np.exp(1j * np.angle(stft))
        noise_estimate = librosa.istft(noise_stft, length=len(signal))
        noise_mask = np.ones(len(signal), dtype=bool)  # All segments considered
    
    else:
        raise ValueError(f"Unknown method: {method}")
    
    return noise_estimate, noise_mask

def detect_empty_segments(signal, sr, frame_length=2048, hop_length=512, quiet_percentile=10, min_segment_duration=0.1):
    """
    Detect almost empty (quiet/silence) segments in a microphone recording.
    Since microphone recordings always have background noise, this detects segments
    that are significantly quieter than speech segments.
    
    Parameters:
    -----------
    signal : array-like
        Audio signal
    sr : int
        Sample rate
    frame_length : int
        Frame length for analysis
    hop_length : int
        Hop length for analysis
    quiet_percentile : float
        Percentile threshold for quiet segments (default 10th percentile)
        Lower values = more sensitive (detects more quiet segments)
    min_segment_duration : float
        Minimum duration in seconds for a segment to be considered (default 0.1s)
        Filters out very brief quiet moments
    
    Returns:
    --------
    empty_mask : array
        Boolean mask indicating quiet/almost empty segments
    empty_segments : list
        List of tuples (start_time, end_time) in seconds for each quiet segment
    """
    signal = np.array(signal)
    frame_energy = []
    frame_times = []
    
    # Calculate frame energy
    for i in range(0, len(signal) - frame_length + 1, hop_length):
        frame = signal[i:i + frame_length]
        energy = np.mean(frame ** 2)
        frame_energy.append(energy)
        frame_times.append(i / sr)  # Time in seconds
    
    frame_energy = np.array(frame_energy)
    frame_times = np.array(frame_times)
    
    if len(frame_energy) == 0 or np.max(frame_energy) == 0:
        # If signal is completely silent
        empty_mask = np.ones(len(signal), dtype=bool)
        empty_segments = [(0, len(signal) / sr)]
        return empty_mask, empty_segments
    
    # Use percentile-based threshold - segments below this are considered "almost empty"
    # This accounts for the fact that microphone recordings always have background noise
    threshold_linear = np.percentile(frame_energy, quiet_percentile)
    
    # Alternative: Use median of lower quartile as threshold (more robust)
    # This identifies segments that are significantly quieter than typical speech
    lower_quartile_energy = frame_energy[frame_energy < np.percentile(frame_energy, 25)]
    if len(lower_quartile_energy) > 0:
        # Use median of quietest 25% as threshold
        threshold_linear = np.median(lower_quartile_energy)
    else:
        threshold_linear = np.percentile(frame_energy, quiet_percentile)
    
    # Identify quiet frames (almost empty segments)
    quiet_frames = frame_energy < threshold_linear
    
    # Convert frame-level mask to sample-level mask
    empty_mask = np.zeros(len(signal), dtype=bool)
    for i, is_quiet in enumerate(quiet_frames):
        start = i * hop_length
        end = min(start + frame_length, len(signal))
        empty_mask[start:end] = is_quiet
    
    # Extract continuous quiet segments (filter by minimum duration)
    empty_segments = []
    in_segment = False
    segment_start = 0
    min_frames = int(min_segment_duration * sr / hop_length)
    segment_start_frame = 0
    
    for i, is_quiet in enumerate(quiet_frames):
        if is_quiet and not in_segment:
            # Start of new quiet segment
            in_segment = True
            segment_start = frame_times[i]
            segment_start_frame = i
        elif not is_quiet and in_segment:
            # End of quiet segment
            in_segment = False
            segment_end = frame_times[i]
            # Only include if segment is long enough
            if (i - segment_start_frame) >= min_frames:
                empty_segments.append((segment_start, segment_end))
    
    # Handle case where segment extends to end
    if in_segment:
        segment_end = len(signal) / sr
        if (len(quiet_frames) - segment_start_frame) >= min_frames:
            empty_segments.append((segment_start, segment_end))
    
    return empty_mask, empty_segments

def calculate_snr_from_single_file(audio_file, method='vad', **kwargs):
    """
    Calculate SNR from a single audio file by estimating noise.
    
    Parameters:
    -----------
    audio_file : str
        Path to audio file
    method : str
        Method to estimate noise: 'vad' or 'spectral_subtraction'
    **kwargs
        Additional parameters for noise estimation
    
    Returns:
    --------
    snr_db : float
        SNR in decibels
    noise_estimate : array
        Estimated noise signal
    noise_mask : array
        Boolean mask indicating noise segments
    empty_mask : array
        Boolean mask indicating empty/almost empty segments
    empty_segments : list
        List of tuples (start_time, end_time) in seconds for each empty segment
    """
    # Load audio file
    signal, sr = librosa.load(audio_file, sr=None)
    
    # Estimate noise
    noise_estimate, noise_mask = estimate_noise_from_signal(signal, sr, method=method, **kwargs)
    
    # Detect almost empty (quiet) segments
    quiet_percentile = kwargs.get('quiet_percentile', 10)
    min_segment_duration = kwargs.get('min_segment_duration', 0.1)
    empty_mask, empty_segments = detect_empty_segments(signal, sr, quiet_percentile=quiet_percentile, min_segment_duration=min_segment_duration)
    
    # Calculate signal power (from speech segments if using VAD)
    if method == 'vad' and np.any(~noise_mask):
        signal_segments = signal[~noise_mask]
        signal_power = np.mean(signal_segments ** 2)
    else:
        signal_power = np.mean(signal ** 2)
    
    # Calculate noise power
    if np.any(noise_mask):
        noise_segments = noise_estimate[noise_mask]
        noise_power = np.mean(noise_segments ** 2) if len(noise_segments) > 0 else np.mean(noise_estimate ** 2)
    else:
        noise_power = np.mean(noise_estimate ** 2)
    
    # Avoid division by zero
    if noise_power == 0:
        snr_db = np.inf
    else:
        snr_db = 10 * np.log10(signal_power / noise_power)
    
    return snr_db, noise_estimate, noise_mask, empty_mask, empty_segments

def calculate_clipping_ratio(signal, threshold=0.99):
    """
    Calculate the clipping ratio of an audio signal.
    
    Clipping occurs when audio samples exceed the maximum representable amplitude,
    causing distortion. This function calculates the percentage of samples that
    are at or near the clipping threshold.
    
    Parameters:
    -----------
    signal : array-like
        Audio signal (assumed to be normalized to [-1, 1])
    threshold : float
        Threshold for considering a sample as clipped (default 0.99)
        Values at or above this absolute value are considered clipped
    
    Returns:
    --------
    clipping_ratio : float
        Percentage of samples that are clipped (0-100)
    num_clipped : int
        Number of clipped samples
    peak_amplitude : float
        Maximum absolute amplitude in the signal
    """
    signal = np.array(signal)
    
    # Count samples at or above threshold (both positive and negative)
    clipped_mask = np.abs(signal) >= threshold
    num_clipped = np.sum(clipped_mask)
    
    # Calculate ratio as percentage
    clipping_ratio = (num_clipped / len(signal)) * 100
    
    # Get peak amplitude
    peak_amplitude = np.max(np.abs(signal))
    
    return clipping_ratio, num_clipped, peak_amplitude

def calculate_clipping_from_file(audio_file, threshold=0.99):
    """
    Calculate clipping ratio from an audio file.
    
    Parameters:
    -----------
    audio_file : str
        Path to audio file
    threshold : float
        Threshold for considering a sample as clipped (default 0.99)
    
    Returns:
    --------
    clipping_ratio : float
        Percentage of samples that are clipped (0-100)
    num_clipped : int
        Number of clipped samples
    peak_amplitude : float
        Maximum absolute amplitude in the signal
    peak_db : float
        Peak amplitude in dB (relative to full scale)
    """
    # Load audio file
    signal, sr = librosa.load(audio_file, sr=None)
    
    clipping_ratio, num_clipped, peak_amplitude = calculate_clipping_ratio(signal, threshold)
    
    # Calculate peak in dB (dBFS - decibels relative to full scale)
    if peak_amplitude > 0:
        peak_db = 20 * np.log10(peak_amplitude)
    else:
        peak_db = -np.inf
    
    return clipping_ratio, num_clipped, peak_amplitude, peak_db

def visualize_segments(signal, sr, noise_mask, empty_mask, empty_segments, width=80):
    """
    Visualize audio segments in terminal with colors.
    
    Parameters:
    -----------
    signal : array-like
        Audio signal
    sr : int
        Sample rate
    noise_mask : array
        Boolean mask for noise segments
    empty_mask : array
        Boolean mask for empty/quiet segments
    empty_segments : list
        List of (start, end) tuples in seconds
    width : int
        Width of visualization in characters
    """
    duration = len(signal) / sr
    
    # ANSI color codes
    RESET = '\033[0m'
    SPEECH = '\033[92m'  # Green
    NOISE = '\033[93m'   # Yellow
    EMPTY = '\033[91m'   # Red
    BOLD = '\033[1m'
    
    # Create timeline visualization
    timeline = []
    for i in range(width):
        # Map character position to time
        t = (i / width) * duration
        sample_idx = int(t * sr)
        if sample_idx >= len(signal):
            sample_idx = len(signal) - 1
        
        if empty_mask[sample_idx]:
            timeline.append(f"{EMPTY}█{RESET}")
        elif noise_mask[sample_idx]:
            timeline.append(f"{NOISE}█{RESET}")
        else:
            timeline.append(f"{SPEECH}█{RESET}")
    
    # Create legend
    legend = f"{BOLD}Legend:{RESET} {SPEECH}█{RESET} Speech  {NOISE}█{RESET} Noise  {EMPTY}█{RESET} Empty/Quiet"
    
    # Format empty segments nicely
    formatted_segments = []
    for start, end in empty_segments:
        formatted_segments.append(f"({float(start):.3f}, {float(end):.3f})")
    
    return ''.join(timeline), legend, formatted_segments


In [2]:
# Calculate SNR from single audio files
dir = "./audio/development/ruken"

# Get all wav files
wav_files = sorted([f for f in os.listdir(dir) if f.endswith(".wav")])

print("Calculating SNR, clipping ratio, and empty segments from single files:\n")
for wav_file in wav_files[:5]:  # Process first 5 files
    file_path = os.path.join(dir, wav_file)
    signal, sr = librosa.load(file_path, sr=None)
    snr, noise_est, noise_mask, empty_mask, empty_segments = calculate_snr_from_single_file(file_path, method='vad')
    
    # Calculate clipping ratio
    clipping_ratio, num_clipped, peak_amplitude, peak_db = calculate_clipping_from_file(file_path)
    
    noise_percentage = np.sum(noise_mask) / len(noise_mask) * 100
    empty_percentage = np.sum(empty_mask) / len(empty_mask) * 100
    
    # Visualize segments
    timeline, legend, formatted_segments = visualize_segments(signal, sr, noise_mask, empty_mask, empty_segments)
    
    # Print results with visualization
    print(f"\n{'='*80}")
    print(f"File: {wav_file}")
    print(f"{'='*80}")
    print(f"SNR: {snr:.2f} dB")
    print(f"Peak amplitude: {peak_amplitude:.4f} ({peak_db:.2f} dBFS)")
    print(f"Clipping ratio: {clipping_ratio:.4f}% ({num_clipped} samples)")
    print(f"Noise segments: {noise_percentage:.1f}%")
    print(f"Empty/almost empty segments: {empty_percentage:.1f}%")
    print(f"Number of empty segments: {len(empty_segments)}")
    print()
    print(timeline)
    print(legend)
    print()
    if formatted_segments:
        print(f"Empty segment times (seconds):")
        for seg in formatted_segments:
            print(f"  {seg}")
    print()


Calculating SNR, clipping ratio, and empty segments from single files:


File: 01.wav
SNR: 17.61 dB
Peak amplitude: 0.7030 (-3.06 dBFS)
Clipping ratio: 0.0000% (0 samples)
Noise segments: 20.8%
Empty/almost empty segments: 12.3%
Number of empty segments: 1

████████████████████████████████████████████████████████████████████████████████
Legend: █ Speech  █ Noise  █ Empty/Quiet

Empty segment times (seconds):
  (0.000, 1.088)


File: 02.wav
SNR: 16.87 dB
Peak amplitude: 0.7143 (-2.92 dBFS)
Clipping ratio: 0.0000% (0 samples)
Noise segments: 20.9%
Empty/almost empty segments: 13.3%
Number of empty segments: 6

████████████████████████████████████████████████████████████████████████████████
Legend: █ Speech  █ Noise  █ Empty/Quiet

Empty segment times (seconds):
  (0.000, 0.608)
  (6.912, 7.040)
  (7.136, 7.296)
  (8.544, 8.704)
  (8.864, 8.960)
  (10.816, 11.049)


File: 03.wav
SNR: 18.04 dB
Peak amplitude: 0.9626 (-0.33 dBFS)
Clipping ratio: 0.0000% (0 samples)
Noise segments: 20.7%
Emp